### Testing for exploding the articles column

In [5]:
import pandas as pd

In [2]:
data = {'age': [59, 18, 63],
        'sample':  [({'0': {'a':'yes', 'b': 'no'}, '1':{'a': 'maybe', 'b': 'NA'}}), ({'0': {'a':'no', 'b': 'yes'}}),  ({'0': {'a':'no', 'b': 'no'}, '1':{'a': 'no', 'b': 'yes'}, '2':{'a': 'yes', 'b': 'yes'}})],
        'date': ['2022-09-29', '2022-08-29', '2022-07-29']}

In [7]:
df = pd.DataFrame.from_dict(data, orient='index', columns=['age', 'sample', 'date'])
df = df.transpose()
df = df.reset_index(drop=True)

In [ ]:
pd.concat(t['sample'].apply(pd.DataFrame).tolist(), 
                        keys=t["age"]).reset_index(level="age")

In [8]:
out = df.drop(columns='sample').join(
 pd.json_normalize(df['sample'])
   .pipe(lambda d: d.set_axis(d.columns.str.split('.', expand=True), axis=1))
   .stack().rename_axis([None, 'new']).reset_index(-1)
)

In [9]:
out

,age,date,new,0,1,2
0,59,2022-09-29,a,yes,maybe,NaN
0,59,2022-09-29,b,no,NA,NaN
1,18,2022-08-29,a,no,NaN,NaN
1,18,2022-08-29,b,yes,NaN,NaN
2,63,2022-07-29,a,no,no,yes
2,63,2022-07-29,b,no,yes,yes


In [10]:
s = df['sample'].map(dict.values).explode()
df.drop(columns=['sample']).join(pd.DataFrame([*s], s.index))

,age,date,a,b
0,59,2022-09-29,yes,no
0,59,2022-09-29,maybe,NA
1,18,2022-08-29,no,yes
2,63,2022-07-29,no,no
2,63,2022-07-29,no,yes
2,63,2022-07-29,yes,yes
